## Azure ML Stable Diffusion Expriment

To use this notebook, you need to download `config.json` file from Azure ML Workspace and place it in this folder. This will allow us to get the workspace reference right away:

First we get the dataset from the local directory 

In [ ]:
#Please change the directory to wherever the art data is stored. Etiher locally or on Azure ML notebooks directory
dataset = r'Ukiyo_e'

Then we have to remove certain keywords from the image names.

In [ ]:
import os
import re
# List of words to remove from filenames
words_to_remove = ["train", "test", "valid"]

# Function to rename files
def rename_files(directory):
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames:
            # Use regular expressions to replace "train," "test," and "valid" in filenames
            new_filename = re.sub(r'(?i)(train|test|valid)', '', filename)
            
            # Check if the filename has changed
            if new_filename != filename:
                old_path = os.path.join(foldername, filename)
                new_path = os.path.join(foldername, new_filename)
                # Rename the file
                os.rename(old_path, new_path)
                print(f'Renamed: {filename} -> {new_filename}')

# Call the function to rename files
rename_files(dataset)

Next we get the Azure ML workspace. 

In [ ]:
#Remeber to download the config file from the Azure Portal for your workspace
from azureml.core import Workspace

try:
    ws = Workspace.from_config()
    print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')
    print('Library configuration succeeded')
except:
    print('Workspace not found')

Now we  get the default datastore in the workspace.

In [ ]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath
ds = ws.get_default_datastore()

And now we upload the images dataset into the Azure ML Workspace's default datastore:

In [ ]:
#You may change the DataPath(ds,"ukiyo_e") to DataPath(ds,<your directory name here>)
Dataset.File.upload_directory(src_dir=dataset, target=DataPath(ds,"ukiyo_e"), overwrite = True, show_progress=True)